In [ ]:
# ============================================
# 1
# Descargar SOLO una carpeta desde GitHub en Google Colab (Sparse Checkout)
# Deja la carpeta lista en /content/DataImg
# ============================================

import os
import shutil
import subprocess

# 1) Variables
REPO_URL = "https://github.com/GuidoRiosCiaffaroni/Machine_Learning_II.git"
FOLDER_NAME = "DataImg/TomatoDataset_ready"
TMP_DIR = "/content/repo_temporal"

# 2) Helpers
def run(cmd, cwd=None):
    print(f"\n$ {cmd}")
    subprocess.check_call(cmd, shell=True, cwd=cwd)

def get_default_branch(repo_url):
    # Detecta rama por defecto (main / master)
    try:
        out = subprocess.check_output(
            f'git ls-remote --symref {repo_url} HEAD',
            shell=True,
            text=True
        )
        for line in out.splitlines():
            if line.startswith("ref:"):
                return line.split("refs/heads/")[-1].split("\t")[0].strip()
    except Exception:
        pass
    return "main"

# 3) Preparación: limpiar previos
shutil.rmtree(TMP_DIR, ignore_errors=True)
shutil.rmtree(f"/content/{FOLDER_NAME}", ignore_errors=True)
os.makedirs(TMP_DIR, exist_ok=True)

# 4) Detectar rama por defecto
BRANCH = get_default_branch(REPO_URL)
print(f" Rama detectada: {BRANCH}")

# 5) Inicializar repo temporal y configurar sparse checkout
run("git init", cwd=TMP_DIR)
run(f'git remote add origin "{REPO_URL}"', cwd=TMP_DIR)
run("git sparse-checkout init --cone", cwd=TMP_DIR)
run(f'git sparse-checkout set "{FOLDER_NAME}"', cwd=TMP_DIR)

# 6) Descargar solo esa carpeta
run(f'git pull --depth 1 origin "{BRANCH}"', cwd=TMP_DIR)

# 7) Mover a /content y limpiar
src_path = os.path.join(TMP_DIR, FOLDER_NAME)
dst_path = os.path.join("/content", FOLDER_NAME)

if not os.path.exists(src_path):
    raise FileNotFoundError(
        f"No se encontró la carpeta '{FOLDER_NAME}' en el repo. "
        f"Revisa que exista en la rama '{BRANCH}'."
    )

shutil.move(src_path, dst_path)
shutil.rmtree(TMP_DIR, ignore_errors=True)

print(f"\n Proceso finalizado. Carpeta lista en: {dst_path}")
print(" Archivos (primeros 30):")
print(os.listdir(dst_path)[:30])


 Rama detectada: main

$ git init

$ git remote add origin "https://github.com/GuidoRiosCiaffaroni/Machine_Learning_II.git"

$ git sparse-checkout init --cone

$ git sparse-checkout set "DataImg/TomatoDataset_ready"

$ git pull --depth 1 origin "main"

 Proceso finalizado. Carpeta lista en: /content/DataImg/TomatoDataset_ready
 Archivos (primeros 30):
['Tomato___Late_blight', 'Tomato___Septoria_leaf_spot', 'Tomato___Tomato_mosaic_virus', 'Tomato___Target_Spot', 'Tomato___Leaf_Mold', 'Tomato___Bacterial_spot', 'Tomato___healthy', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Early_blight', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus']


# Colab — KNN para enfermedades del tomate (con embeddings CNN)

# Bloque 1 — Imports y reproducibilidad



## Objetivo del bloque
Este bloque prepara el **entorno de trabajo** para el proyecto de clasificación de imágenes, asegurando que:
1. Las **librerías necesarias** estén disponibles (manejo de archivos, análisis de datos, Deep Learning y Machine Learning).
2. La ejecución sea **reproducible**, es decir, que al correr el notebook múltiples veces se obtengan resultados **consistentes** (dentro de lo posible).

---

## Paso 1: Importación de librerías base (sistema y utilidades)
Se cargan librerías de propósito general para:
- **Navegar por directorios y manejar rutas** (útil para datasets organizados en carpetas por clase).
- **Listar archivos de imágenes** con patrones de búsqueda (por ejemplo, `*.jpg`, `*.png`).
- Controlar la **aleatoriedad** del proceso experimental (por ejemplo, particiones de datos).

**Relevancia:** en clasificación de imágenes con estructura “carpetas = clases”, este conjunto de herramientas es esencial para construir un índice de imágenes y etiquetas.

---

## Paso 2: Importación de librerías científicas y de análisis
Se importan herramientas estándar para:
- **Cálculo numérico** y operaciones vectorizadas (base para features, embeddings y métricas).
- **Manipulación tabular** de datos (ideal para construir un *dataframe* que contenga `path` de la imagen y `label`).

**Relevancia:** aunque el problema sea de imágenes, mantener un registro estructurado en forma de tabla facilita:
- auditoría del dataset,
- conteo por clase,
- detección de desbalance,
- trazabilidad del pipeline.

---

## Paso 3: Importación de TensorFlow (Deep Learning)
Se incorpora TensorFlow como framework para:
- Cargar imágenes eficientemente (por lotes).
- Aplicar preprocesamiento y/o extracción de características (embeddings) usando redes preentrenadas.
- Estandarizar el flujo de trabajo de visión por computador en Colab.

**Relevancia:** TensorFlow permite manejar imágenes a escala y aprovechar aceleración por GPU.

---

## Paso 4: Importación de herramientas de scikit-learn (ML clásico y evaluación)
Se importan módulos clave para construir un pipeline reproducible y evaluable:

### 4.1 Partición y validación
- División de datos en conjuntos de entrenamiento y prueba.
- Estrategias de validación cruzada estratificada (importante en clasificación multiclase desbalanceada).
- Búsqueda sistemática de hiperparámetros (*Grid Search*).

**Relevancia:** asegura una evaluación correcta de generalización y evita conclusiones basadas en un solo split.

### 4.2 Preprocesamiento
- Codificación de etiquetas (transformar nombres de clases a números).
- Estandarización de características (crítica para algoritmos basados en distancia como KNN).

**Relevancia:** KNN depende directamente de distancias; si las variables no están en escalas comparables, el modelo se sesga.

### 4.3 Pipeline
Se utiliza un enfoque de “pipeline” para encadenar:
- transformaciones (ej. escalamiento),
- modelo (KNN),
de forma consistente y sin fugas de información (*data leakage*).

**Relevancia:** mejora buenas prácticas y hace el experimento replicable y ordenado.

### 4.4 Métricas
Se incluyen métricas para evaluación robusta:
- **Accuracy** (visión global).
- **F1-score** (balance precisión/recall, especialmente útil en desbalance).
- **Reporte por clase** (precision, recall, F1 y soporte).
- **Matriz de confusión** (diagnóstico de confusiones entre enfermedades).

**Relevancia:** en problemas multiclase de enfermedades, es crucial analizar **qué clases se confunden** para justificar mejoras.

---

## Paso 5: Definición de semilla y control de aleatoriedad (reproducibilidad)
Se fija una semilla global (`SEED = 42`) y se aplica a:
- Generador aleatorio estándar,
- generador de NumPy,
- generador de TensorFlow.

**Por qué es importante:**
- Reduce variabilidad entre ejecuciones.
- Permite reproducir splits, resultados de entrenamiento y comparaciones de modelos de forma más confiable.
- Es un requisito típico en contextos académicos y profesionales.

> Nota metodológica: en Deep Learning, la reproducibilidad total puede depender también del hardware (GPU) y de operaciones no deterministas, pero fijar semillas sigue siendo una práctica esencial.

---

## Paso 6: Verificación del entorno
Finalmente, se imprime una confirmación de que el entorno está listo, incluyendo la versión de TensorFlow.

**Relevancia:**
- Asegura trazabilidad (por ejemplo, para comparar resultados entre notebooks o máquinas).
- Es útil para depuración y documentación del informe.

---

## Resultado del bloque
Al finalizar este bloque, el notebook queda preparado para:
1. Indexar el dataset de imágenes desde carpetas.
2. Preprocesar etiquetas y particionar datos de manera consistente.
3. Entrenar modelos con evaluación y ajuste de hiperparámetros.
4. Mantener un flujo de trabajo reproducible y defendible.


In [ ]:
# ============================================
# Bloque 1: Imports y reproducibilidad
# ============================================

import os
import glob
import random
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix,
    accuracy_score, f1_score
)

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("✅ Entorno listo. TF:", tf.__version__)


✅ Entorno listo. TF: 2.19.0


# Bloque 2 — Ruta del dataset y detección de clases



## Objetivo del bloque
Este bloque establece el **punto de entrada al conjunto de datos** y construye el listado oficial de **clases** del problema de clasificación, asumiendo una estructura supervisada típica:

- **Una carpeta por clase** (enfermedad o estado saludable).
- Las imágenes correspondientes se encuentran dentro de cada carpeta.

El resultado es un vector `class_names` que actúa como referencia para:
- la codificación de etiquetas,
- la creación del índice de imágenes,
- la interpretación de métricas por clase (reporte y matriz de confusión),
- y la consistencia del pipeline completo.

---

## Paso 1: Definición de la ruta raíz del dataset
Se define una variable que apunta al directorio donde está almacenado el dataset en Google Colab.

**Relevancia práctica**
- Centraliza la configuración: si cambia la ubicación del dataset, basta modificar esta línea.
- Facilita reutilización del notebook y evita “rutas duras” dispersas en el código.

**Relevancia metodológica**
- Permite que el pipeline sea reproducible: el dataset se localiza desde un único punto controlado.

---

## Paso 2: Validación temprana de existencia del directorio
Se utiliza una verificación inmediata (assert) para asegurar que la ruta definida **existe realmente**.

**Por qué es importante**
- Evita errores posteriores más difíciles de depurar (por ejemplo, listas vacías o fallas al leer imágenes).
- Implementa una buena práctica de ingeniería: *fail fast* (fallar temprano si una condición crítica no se cumple).
- Asegura que el notebook sea autocontenido: informa claramente cuándo el problema es de ruta/montaje y no del modelo.

---

## Paso 3: Identificación automática de clases a partir de carpetas
Se recorre el contenido del directorio raíz y se seleccionan únicamente los elementos que son **subdirectorios**, ya que cada uno representa una clase del problema.

**Qué significa esto en términos de ML supervisado**
- Cada carpeta funciona como la “etiqueta” de sus imágenes.
- Por ejemplo, una carpeta `Tomato___Late_blight` representa la clase *Late blight*.
- Esta convención es estándar en clasificación de imágenes y se alinea con loaders comunes en frameworks de visión.

---

## Paso 4: Ordenamiento de clases para consistencia
El listado de clases se ordena alfabéticamente.

**Por qué el orden importa**
- Garantiza que el mapeo clase → índice sea **estable** entre ejecuciones.
- Evita inconsistencias del tipo: “la clase 0 cambió” entre distintos runs o notebooks.
- Asegura trazabilidad en reportes y matrices de confusión, donde el orden de clases debe ser fijo.

---

## Paso 5: Reporte de control (sanity check)
Finalmente, se imprime:
- el número total de clases detectadas,
- y el listado completo de nombres de clases.

**Relevancia**
- Confirma que el dataset se está interpretando correctamente.
- Permite detectar problemas frecuentes:
  - carpetas extra (por ejemplo, `.ipynb_checkpoints`),
  - errores de estructura (clases dentro de una carpeta adicional),
  - o rutas mal definidas.

---

## Resultado del bloque
Al finalizar este bloque se obtiene:
1. Un **directorio raíz validado** (`DATA_ROOT`) desde donde se leerán las imágenes.
2. Un listado consistente y ordenado de **clases** (`class_names`), que será la base para:
   - construir el dataset (`path`, `label`),
   - codificar etiquetas numéricas,
   - y evaluar el desempeño por clase de forma correcta.


In [ ]:
# ============================================
# Bloque 2: Ruta del dataset y clases
# ============================================

DATA_ROOT = "/content/DataImg/TomatoDataset_ready"  # ajusta si tu ruta difiere

assert os.path.exists(DATA_ROOT), f"❌ No existe la ruta: {DATA_ROOT}"

class_names = sorted([d for d in os.listdir(DATA_ROOT) if os.path.isdir(os.path.join(DATA_ROOT, d))])
print("✅ Clases encontradas:", len(class_names))
print(class_names)



✅ Clases encontradas: 10
['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']


# Bloque 3 — Indexación de imágenes por clase



## Objetivo del bloque
Este bloque construye el **catálogo estructurado del dataset** a partir de la organización “carpeta = clase”.  
En términos prácticos, transforma una colección de archivos (imágenes distribuidas en subcarpetas) en una tabla con dos columnas fundamentales:

- **`path`**: ruta completa del archivo de imagen.
- **`label`**: nombre de la clase asociada (la carpeta donde está la imagen).

Este paso es esencial porque permite tratar el dataset como un conjunto de datos supervisado estándar, facilitando el preprocesamiento, el *split* estratificado, la auditoría del desbalance y la trazabilidad del pipeline.

---

## Paso 1: Definición de extensiones válidas de imagen
Se establece un conjunto de extensiones admitidas (por ejemplo `.jpg`, `.jpeg`, `.png`) para filtrar únicamente archivos que correspondan a imágenes.

**Por qué es importante**
- Evita incluir archivos no deseados (por ejemplo, `txt`, `json`, miniaturas o archivos del sistema).
- Asegura que el pipeline de lectura posterior no falle por tipos de archivo incompatibles.

---

## Paso 2: Inicialización de contenedores para rutas y etiquetas
Se crean dos listas vacías:
- una para acumular las **rutas de las imágenes**,
- otra para acumular la **etiqueta (clase)** de cada imagen.

**Interpretación supervisada**
- Cada elemento de `image_paths` representa una instancia \(x_i\).
- Cada elemento correspondiente en `labels` representa la etiqueta \(y_i\).
- En conjunto, se construye el mapeo \((x_i, y_i)\) requerido por aprendizaje supervisado.

---

## Paso 3: Recorrido clase por clase (carpeta por carpeta)
Se itera sobre el listado de clases detectadas previamente. Para cada clase:

1. Se construye la ruta de su carpeta.
2. Se buscan todos los archivos de imagen dentro de esa carpeta que coincidan con las extensiones válidas.
3. Se agregan las rutas encontradas al catálogo global.
4. Se generan las etiquetas correspondientes repitiendo el nombre de la clase tantas veces como imágenes se encontraron.

**Fundamento**
- La **etiqueta** se deriva de la carpeta: esto implementa la convención estándar en visión por computador para clasificación supervisada.
- Se conserva trazabilidad: cada imagen queda asociada explícitamente a una clase.

---

## Paso 4: Creación de una tabla estructurada (DataFrame)
Las dos listas (rutas y etiquetas) se convierten en una tabla de datos con dos columnas:

- `path`: ubicaciones de los archivos
- `label`: clase de cada imagen

**Ventajas metodológicas**
- Permite aplicar operaciones de análisis exploratorio (conteos, muestreos, filtros).
- Facilita el *split* estratificado porque las etiquetas quedan explícitas.
- Ayuda a detectar errores en el dataset (clases vacías, rutas inválidas, etc.).

---

## Paso 5: Verificación inicial del dataset (sanity check)
Se imprime el **número total de imágenes** y se muestra una vista previa de las primeras filas.

**Por qué es importante**
- Confirma que el índice fue construido correctamente.
- Permite verificar rápidamente:
  - rutas bien formadas,
  - correspondencia correcta entre rutas y etiquetas,
  - y ausencia de listas vacías (lo que indicaría una ruta mal definida o carpetas sin imágenes).

---

## Paso 6: Análisis de distribución por clase (detección de desbalance)
Se calcula el conteo de imágenes por clase y se ordena (normalmente de menor a mayor) para identificar:

- clases minoritarias,
- clases dominantes,
- posibles problemas de “clase con pocas muestras”.

**Relevancia para Machine Learning**
- En clasificación multiclase, el **desbalance** afecta fuertemente:
  - la interpretación de *accuracy*,
  - el rendimiento en clases minoritarias,
  - y la capacidad del modelo para generalizar.
- Este diagnóstico justifica decisiones posteriores, por ejemplo:
  - ajustar métricas (Macro F1),
  - aplicar técnicas de balanceo o augmentación,
  - o filtrar clases con muy pocas muestras (si son errores del dataset).

---

## Resultado del bloque
Al finalizar este bloque se obtiene:

1. Un **dataset supervisado estructurado** en forma de tabla (`df`) con pares \((x, y)\).
2. Un conteo por clase que permite evaluar la **calidad y balance** del conjunto de datos.
3. Un punto de control crítico para prevenir errores posteriores en el *split* estratificado o en la evaluación.


In [ ]:
# ============================================
# Bloque 3: Indexar imágenes por clase
# ============================================

valid_ext = (".jpg", ".jpeg", ".png")

image_paths, labels = [], []

for cls in class_names:
    cls_dir = os.path.join(DATA_ROOT, cls)
    files = []
    for ext in valid_ext:
        files.extend(glob.glob(os.path.join(cls_dir, f"*{ext}")))
    image_paths.extend(files)
    labels.extend([cls] * len(files))

df = pd.DataFrame({"path": image_paths, "label": labels})

print("✅ Total imágenes:", len(df))
display(df.head())

counts = df["label"].value_counts().sort_values(ascending=True)
print("✅ Distribución por clase:")
display(counts)



✅ Total imágenes: 150


,path,label
0,/content/DataImg/TomatoDataset_ready/Tomato___...,Tomato___Late_blight
1,/content/DataImg/TomatoDataset_ready/Tomato___...,Tomato___Late_blight
2,/content/DataImg/TomatoDataset_ready/Tomato___...,Tomato___Late_blight
3,/content/DataImg/TomatoDataset_ready/Tomato___...,Tomato___Late_blight
4,/content/DataImg/TomatoDataset_ready/Tomato___...,Tomato___Late_blight


✅ Distribución por clase:


,count
label,
Tomato___healthy,1
Tomato___Late_blight,149


# Bloque 4 — Diagnóstico y filtrado de clases con pocas muestras



## Objetivo del bloque
Este bloque realiza un **control de calidad del dataset** para asegurar que cada clase tenga suficientes muestras como para permitir:

- **partición estratificada** (train/val/test),
- **evaluación estable** con métricas por clase,
- y **entrenamiento confiable** sin clases “degeneradas”.

En clasificación supervisada multiclase, si alguna clase tiene muy pocas imágenes (por ejemplo 1 o 2), se producen dos problemas críticos:

1. **Imposibilidad de estratificar correctamente**: el algoritmo de partición no puede garantizar que todas las clases estén representadas en train y test (o val).
2. **Métricas inválidas o inestables**: el reporte por clase (precision/recall/F1) pierde sentido estadístico cuando el soporte es extremadamente bajo.

---

## Paso 1: Diagnóstico del mínimo de muestras por clase
Se identifica el tamaño de la clase más pequeña (el mínimo conteo dentro del vector de frecuencias por clase).

**Interpretación**
- Si el mínimo es **1**, es imposible aplicar `stratify` en un split estándar porque no hay forma de distribuir esa clase en más de un conjunto.
- Si el mínimo es bajo (por ejemplo 2–4), incluso si el split se ejecuta, la evaluación será frágil y altamente variable.

Este diagnóstico justifica decisiones posteriores de filtrado o recolección de más datos.

---

## Paso 2: Definición del umbral mínimo por clase (criterio de filtrado)
Se fija un parámetro `MIN_PER_CLASS` que determina la cantidad mínima de imágenes requeridas para que una clase permanezca en el experimento.

**Justificación metodológica del umbral**
- Un valor como **5** es un estándar razonable para permitir particiones múltiples con estratificación (train/val/test).
- Mientras más alta la exigencia (por ejemplo 10 o 20), mayor estabilidad estadística, pero también se eliminan más clases o datos.
- Este umbral es un compromiso entre **calidad del experimento** y **cobertura del conjunto de clases**.

---

## Paso 3: Selección de clases válidas según el umbral
Se filtra el conjunto de clases, conservando únicamente aquellas cuyo conteo es mayor o igual al mínimo definido.

**Resultado conceptual**
Se construye el conjunto de clases que tienen suficiente evidencia empírica como para participar en un experimento supervisado válido.

---

## Paso 4: Filtrado del DataFrame para construir el dataset final
Se crea un nuevo dataset (una copia filtrada del DataFrame) que contiene únicamente imágenes pertenecientes a clases válidas.

**Por qué se usa una copia**
- Evita modificar el dataset original (buena práctica para trazabilidad).
- Permite comparar “antes vs después” y auditar qué se eliminó.

---

## Paso 5: Comparación del dataset antes y después del filtrado
Se reporta:

- tamaño total del dataset (número de imágenes),
- cantidad de clases,
antes y después del proceso.

**Interpretación**
- Permite cuantificar el impacto del filtrado:
  - cuántas imágenes se removieron,
  - cuántas clases se excluyeron,
  - y si el experimento sigue siendo representativo del problema.

---

## Paso 6: Identificación explícita de clases eliminadas
Se calcula y reporta el conjunto de clases que fueron removidas por no cumplir el umbral mínimo.

**Relevancia académica**
Esto es clave para el informe y el análisis crítico, porque:
- documenta decisiones metodológicas,
- evita sesgos ocultos (“parece que el modelo funciona bien” porque se eliminaron clases difíciles),
- y permite proponer trabajo futuro (p.ej., “recolectar más imágenes para las clases eliminadas”).

---

## Paso 7: Re-cálculo de la distribución por clase en el dataset filtrado
Se vuelve a calcular la distribución por clase usando el dataset filtrado.

**Por qué es importante**
- Confirma que el filtrado se aplicó correctamente.
- Permite verificar si persiste el desbalance (probablemente sí).
- Establece el nuevo mínimo por clase, validando que ahora se cumple el criterio para splits estratificados.

---

## Resultado del bloque
Al finalizar este bloque se obtiene un dataset depurado (`df_f`) que:

1. **Elimina clases con evidencia insuficiente**, evitando errores de estratificación y métricas inestables.
2. Deja una distribución más adecuada para entrenamiento y evaluación.
3. Genera trazabilidad explícita para el informe: qué se removió y por qué.

Este paso mejora la **validez estadística** del experimento y la **robustez metodológica** del pipeline completo.


In [ ]:
# ============================================
# Bloque 4: Diagnóstico y filtrado de clases con pocas muestras
# ============================================

min_count = counts.min()
print("Mínimo de imágenes en una clase:", min_count)

# Ajusta según tu necesidad:
# - 5 es un umbral razonable si quieres train/val/test estratificado.
MIN_PER_CLASS = 5

valid_classes = counts[counts >= MIN_PER_CLASS].index
df_f = df[df["label"].isin(valid_classes)].copy()

print("\n✅ Dataset antes:", df.shape, " | después:", df_f.shape)
print("✅ Clases antes:", df['label'].nunique(), " | después:", df_f['label'].nunique())

# Reportar clases eliminadas (si las hay)
removed = set(df["label"].unique()) - set(df_f["label"].unique())
if removed:
    print("\n⚠️ Clases removidas por tener < MIN_PER_CLASS imágenes:")
    print(sorted(list(removed)))

# Nueva distribución
counts_f = df_f["label"].value_counts().sort_values(ascending=True)
print("\n✅ Nueva distribución (filtrada):")
display(counts_f)
print("Mínimo ahora:", counts_f.min())



Mínimo de imágenes en una clase: 1

✅ Dataset antes: (150, 2)  | después: (149, 2)
✅ Clases antes: 2  | después: 1

⚠️ Clases removidas por tener < MIN_PER_CLASS imágenes:
['Tomato___healthy']

✅ Nueva distribución (filtrada):


,count
label,
Tomato___Late_blight,149


Mínimo ahora: 149


# Bloque 5 — Codificación de etiquetas (*Label Encoding*) + *split* estratificado


## Objetivo del bloque
Este bloque transforma el dataset filtrado (`df_f`) en un conjunto de datos supervisado listo para modelado, realizando dos tareas fundamentales:

1. **Codificar las etiquetas de clase** (texto → números) para que sean compatibles con algoritmos de Machine Learning y Deep Learning.
2. Construir una partición **estratificada** en tres subconjuntos: **entrenamiento**, **validación** y **prueba**, preservando la proporción relativa de clases en cada subconjunto.

Esto asegura una evaluación metodológicamente correcta y comparaciones justas entre modelos.

---

## Paso 1: Inicialización del codificador de etiquetas
Se instancia un codificador de etiquetas (*LabelEncoder*), cuyo propósito es mapear cada clase categórica (por ejemplo, nombres de enfermedades) a un identificador numérico entero.

**Motivación**
- Los modelos no operan directamente con cadenas de texto como etiquetas.
- La codificación crea una representación numérica consistente y reversible:
  - clase → índice (entrenamiento/evaluación),
  - índice → clase (interpretación de resultados).

---

## Paso 2: Ajuste del codificador y transformación de etiquetas
Se aplica el proceso de *fit + transform* sobre la columna `label`:

- **fit**: aprende el conjunto de clases disponibles en el dataset filtrado.
- **transform**: asigna un entero a cada clase y genera la variable objetivo numérica `y`.

**Resultado**
- Se crea una nueva columna `y` en el DataFrame, que corresponde a la **variable objetivo** del problema supervisado.

---

## Paso 3: Definición formal de variables de entrada y salida
Se separan explícitamente:

- **X**: variable de entrada, que en este caso corresponde a las **rutas de archivo** de las imágenes.
- **y**: variable objetivo, que corresponde a la **clase numérica** (enfermedad/healthy) asociada a cada imagen.

**Nota metodológica importante**
- Aunque `X` contiene rutas y no pixeles, este enfoque es correcto porque el pipeline posterior cargará las imágenes desde esas rutas de forma eficiente (por lotes) para extraer características o entrenar el modelo.

---

## Paso 4: Primera partición estratificada (Train/Test 80/20)
Se realiza una división en dos conjuntos:

- **Train (80%)**: usado para entrenamiento del modelo y ajuste de hiperparámetros.
- **Test (20%)**: usado únicamente al final para medir generalización.

Se aplica **estratificación** usando `y`, lo que garantiza que la proporción de clases en el conjunto de prueba sea similar a la del dataset original.

**Por qué estratificar**
- En problemas multiclase con posible desbalance, un split aleatorio puede dejar clases poco representadas (o incluso ausentes) en test.
- Estratificar aumenta la validez de las métricas y reduce la varianza del proceso de evaluación.

---

## Paso 5: Segunda partición estratificada (Validación desde Train)
Luego, desde el conjunto de entrenamiento se separa un subconjunto de **validación**.

- Se toma una fracción del train equivalente a **12.5%**, lo que produce aproximadamente **10% del total** como validación.
- La validación se utiliza para:
  - monitorear desempeño durante desarrollo,
  - comparar configuraciones,
  - realizar verificaciones previas a la evaluación final.

Nuevamente se aplica estratificación, esta vez sobre `y_train`, para mantener la distribución de clases dentro de train y val.

---

## Paso 6: Verificación del resultado del split
Se reporta el tamaño de cada conjunto (train/val/test) y el número final de clases.

**Relevancia**
- Verifica que la partición sea consistente con el diseño experimental.
- Confirma cuántas clases sobrevivieron al filtrado previo (`len(le.classes_)`).
- Este control facilita detectar errores como:
  - splits incorrectos,
  - pérdida de clases,
  - o inconsistencias en el dataset.

---

## Resultado del bloque
Al finalizar este bloque se obtiene:

1. Una variable objetivo numérica (`y`) consistente con las clases del dataset.
2. Tres conjuntos estratificados:
   - **Train**: para entrenamiento y tuning,
   - **Val**: para validación intermedia,
   - **Test**: para evaluación final (generalización).
3. Un mapeo estable entre índices y nombres de clases (`le.classes_`), crucial para interpretar reportes y matrices de confusión.


In [ ]:
# ============================================
# Bloque 5: Label Encoding + split estratificado
# ============================================

le = LabelEncoder()
df_f["y"] = le.fit_transform(df_f["label"])

X = df_f["path"].values
y = df_f["y"].values

# 80/20 train-test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=SEED,
    stratify=y
)

# Val desde train (12.5% del train = 10% total aprox)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train,
    test_size=0.125,
    random_state=SEED,
    stratify=y_train
)

print("✅ Split completo:")
print("Train:", len(X_train), "Val:", len(X_val), "Test:", len(X_test))
print("Clases finales:", len(le.classes_))


✅ Split completo:
Train: 104 Val: 15 Test: 30
Clases finales: 1


# Bloque 6 — Construcción del *pipeline* `tf.data`



## Objetivo del bloque
Este bloque implementa un *pipeline* de entrada de datos utilizando la API `tf.data` de TensorFlow, con el propósito de:

1. **Cargar imágenes desde disco** a partir de sus rutas (`paths`).
2. Aplicar **preprocesamiento básico** (decodificación, redimensionamiento y conversión de tipo).
3. Entregar los datos al modelo de manera **eficiente y escalable**, mediante:
   - procesamiento en paralelo,
   - agrupación por lotes (*batching*),
   - y prelectura (*prefetch*) para optimizar el rendimiento en CPU/GPU.

El resultado final son tres datasets (`ds_train`, `ds_val`, `ds_test`) listos para entrenamiento y evaluación.

---

## Paso 1: Definición de hiperparámetros del pipeline (tamaño e intensidad de carga)
Se define:

- **Tamaño objetivo de imagen**: se estandariza cada imagen a una resolución fija.
- **Tamaño de lote (batch size)**: número de imágenes procesadas en conjunto por iteración.

### Justificación técnica
- Un tamaño fijo de imagen es obligatorio para alimentar una red neuronal de forma consistente.
- El tamaño del batch controla el equilibrio entre:
  - velocidad (batches más grandes suelen ser más eficientes),
  - consumo de memoria (batches grandes pueden saturar la RAM/GPU),
  - estabilidad de entrenamiento (aunque aquí el objetivo principal es estandarizar el flujo).

En este pipeline, la resolución seleccionada es coherente con modelos preentrenados comunes (por ejemplo, EfficientNetB0).

---

## Paso 2: Función de carga y preprocesamiento por muestra
Se define una función que recibe:
- una ruta de archivo (path),
- su etiqueta (label),

y retorna la imagen ya procesada junto con su etiqueta.

Dentro de esta función se realizan operaciones estándar:

1. **Lectura del archivo desde disco**: se carga el contenido binario.
2. **Decodificación de la imagen**: se interpreta el binario como imagen RGB (3 canales).
3. **Redimensionamiento** a un tamaño fijo: esto normaliza la entrada al modelo.
4. **Conversión a tipo float**: deja la imagen en un formato numérico compatible con operaciones posteriores (por ejemplo, normalización o *preprocess_input* de un modelo preentrenado).

### Relevancia metodológica
Separar esta lógica en una función:
- hace el flujo modular y reutilizable,
- permite mantener consistencia de preprocesamiento entre train/val/test,
- y facilita cambios (por ejemplo, agregar augmentación solo en entrenamiento).

---

## Paso 3: Función constructora del dataset (`make_dataset`)
Se define una función que arma el dataset completo a partir de:
- una lista/array de rutas,
- una lista/array de etiquetas,
- y una opción de mezcla aleatoria (*shuffle*).

Esta función construye una secuencia de transformaciones:

### 3.1 Creación del dataset desde tensores
Se construye un dataset base emparejando (ruta, etiqueta) como pares \((x_i, y_i)\).

**Ventaja**
- Permite tratar el flujo como un “stream” de datos, sin cargar todo en memoria como imágenes.

### 3.2 Mezcla opcional (*shuffle*)
Si se activa, se mezclan las muestras antes de generar los lotes.

**Justificación**
- En entrenamiento clásico, *shuffle* ayuda a reducir correlaciones y mejorar la generalización.
- En este notebook, mantener `shuffle=False` puede ser útil cuando el objetivo es solo extraer embeddings de manera determinista.
- En escenarios de entrenamiento de redes, normalmente se recomienda `shuffle=True` solo para el conjunto de entrenamiento.

### 3.3 Mapeo del preprocesamiento (`map`)
Se aplica la función de carga y preprocesamiento a cada elemento, habilitando procesamiento paralelo.

**Justificación**
- El parámetro de paralelización permite aprovechar múltiples hilos CPU.
- Mejora significativamente el rendimiento cuando hay muchas imágenes.

### 3.4 Agrupación en lotes (`batch`)
Se agrupan muestras para que el modelo procese múltiples imágenes por iteración.

**Justificación**
- Reduce overhead de llamadas al modelo.
- Aumenta eficiencia computacional, especialmente con GPU.

### 3.5 Prelectura (`prefetch`)
Se habilita prelectura asincrónica de batches.

**Por qué es importante**
- Mientras la GPU procesa un batch, el siguiente se prepara en paralelo.
- Reduce tiempos muertos de entrenamiento/evaluación (pipeline más “fluido”).

---

## Paso 4: Construcción de `ds_train`, `ds_val` y `ds_test`
Se aplica `make_dataset` a los tres subconjuntos resultantes del *split*:

- **Train**: datos para ajustar el modelo.
- **Validation**: datos para monitoreo y decisiones metodológicas.
- **Test**: datos reservados para evaluación final de generalización.

### Importancia metodológica
Separar estos datasets es clave para:
- evitar *data leakage*,
- obtener métricas confiables,
- y cumplir buenas prácticas de evaluación supervisada.

---

## Resultado del bloque
Al finalizar se obtiene un pipeline `tf.data` listo y eficiente que:

1. Carga imágenes desde sus rutas sin saturar memoria.
2. Estandariza el tamaño y formato de entrada.
3. Acelera el flujo mediante paralelización y prefetch.
4. Entrega datasets separados (train/val/test) consistentes con un diseño experimental riguroso.


In [ ]:
# ============================================
# Bloque 6: tf.data pipeline
# ============================================

IMG_SIZE = (224, 224)
BATCH_SIZE = 64

def load_and_preprocess(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32)
    return img, label

def make_dataset(paths, labels, shuffle=False):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(paths), seed=SEED, reshuffle_each_iteration=True)
    ds = ds.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds

ds_train = make_dataset(X_train, y_train, shuffle=False)
ds_val   = make_dataset(X_val, y_val, shuffle=False)
ds_test  = make_dataset(X_test, y_test, shuffle=False)

print("✅ tf.data listo")


✅ tf.data listo


# Bloque 7 — Extractor de *embeddings* mediante *Transfer Learning*



## Objetivo del bloque
Este bloque transforma cada imagen del dataset en un **vector numérico de características** (*embedding*) utilizando una red neuronal convolucional (CNN) preentrenada.  
El propósito es construir una representación compacta y semánticamente útil de las imágenes para posteriormente aplicar un modelo de Machine Learning clásico (por ejemplo, **KNN**) sobre estos vectores.

En términos generales, el bloque implementa el esquema:

**Imagen (pixeles) → CNN preentrenada (sin entrenamiento) → Embedding (vector) → Modelo ML**

---

## Paso 1: Selección de un modelo preentrenado y su función en el pipeline
Se emplea una CNN conocida por su buen rendimiento en visión por computador (EfficientNetB0), entrenada previamente sobre **ImageNet**, un conjunto masivo y diverso de imágenes.

### Justificación teórica (Transfer Learning)
- ImageNet permite que la red aprenda patrones visuales generales (bordes, texturas, manchas, formas).
- Estos patrones suelen ser transferibles a tareas nuevas, como identificación de enfermedades foliares.
- En lugar de entrenar una red desde cero (costoso en datos y cómputo), se reutiliza el conocimiento aprendido como **extractor de características**.

---

## Paso 2: Configuración de la red como extractor (sin “cabeza” de clasificación)
La red se instancia con tres configuraciones clave:

1. **`weights="imagenet"`**  
   Indica que se utilizan pesos ya entrenados.

2. **`include_top=False`**  
   Se elimina la capa final original diseñada para clasificar ImageNet (1000 clases).  
   Esto es fundamental porque nuestro problema tiene clases distintas (enfermedades del tomate).

3. **`pooling="avg"`**  
   Se aplica *Global Average Pooling* al final del modelo para convertir los mapas de activación en un **vector** de tamaño fijo.  
   Esto produce un embedding compacto que:
   - resume la información visual relevante,
   - y es adecuado para modelos basados en distancia o modelos lineales.

**Resultado conceptual:** la CNN se convierte en una función:
\[
f(\text{imagen}) \rightarrow \mathbb{R}^d
\]
donde \( d \) es la dimensión del embedding.

---

## Paso 3: Congelamiento de parámetros (no entrenamiento)
Se desactiva el entrenamiento de la red base.

### Implicancia metodológica
- La CNN no ajusta sus pesos con el dataset de tomate.
- Se utiliza únicamente como una transformación fija: “de imagen a embedding”.
- Esto reduce:
  - tiempo de entrenamiento,
  - riesgo de sobreajuste con datasets pequeños,
  - y complejidad computacional.

---

## Paso 4: Definición de una función optimizada para obtener embeddings por batch
Se define una función que procesa un lote de imágenes y devuelve los embeddings.

Dentro de esta función se aplican dos operaciones esenciales:

1. **Preprocesamiento específico del modelo (`preprocess_input`)**  
   Ajusta escala y distribución de pixeles para que la entrada sea compatible con lo que EfficientNet espera.  
   Esto es crítico porque los modelos preentrenados requieren una normalización consistente con su entrenamiento original.

2. **Inferencia en modo no-entrenamiento**  
   Se fuerza el modo de inferencia para garantizar comportamiento estable (por ejemplo, evitando efectos de capas que se comportan distinto en entrenamiento e inferencia).

Además, la función se marca para ejecución optimizada por TensorFlow, lo que puede mejorar el rendimiento al ejecutar repetidamente sobre muchos batches.

---

## Paso 5: Extracción de embeddings para un dataset completo (train/val/test)
Se define una rutina que recorre un `tf.data.Dataset` por lotes y construye:

- una matriz de embeddings (features),
- un vector de etiquetas correspondientes.

### Qué ocurre en cada iteración del dataset
Para cada batch:
1. Se obtienen las imágenes y sus etiquetas.
2. Se calculan los embeddings del batch con la CNN preentrenada.
3. Se almacenan embeddings y etiquetas en contenedores acumulativos.

Al finalizar:
- los embeddings se concatenan en una única matriz:
  \[
  X \in \mathbb{R}^{n \times d}
  \]
- las etiquetas se concatenan en un vector:
  \[
  y \in \{0, 1, \dots, K-1\}^{n}
  \]

---

## Paso 6: Construcción final de conjuntos embebidos (train/val/test)
Se aplica la extracción a cada partición:

- **Train embeddings**: se usarán para entrenar y ajustar el modelo de ML.
- **Validation embeddings**: se usan para validación intermedia (si corresponde).
- **Test embeddings**: se reservan para evaluación final de generalización.

### Importancia metodológica
Mantener la separación train/val/test también en el espacio de embeddings evita fuga de información y asegura una evaluación justa.

---

## Paso 7: Verificación de dimensiones como control de calidad
Se imprime la forma (*shape*) de los embeddings generados en cada conjunto.

**Por qué esto es clave**
- Confirma que el pipeline produce vectores de dimensión fija.
- Verifica consistencia entre train/val/test.
- Permite detectar errores tempranos (por ejemplo, embeddings vacíos, batches mal cargados o discrepancias de tamaño).

---

## Resultado del bloque
Al terminar este bloque se obtiene:

1. Una representación numérica de las imágenes en forma de embeddings.
2. Tres matrices listas para Machine Learning clásico:
   - \(X_{\text{train}}, X_{\text{val}}, X_{\text{test}}\)
3. Un puente metodológico sólido entre Deep Learning y ML clásico:
   - **Deep Learning** aporta extracción de características,
   - **KNN** (u otros modelos) realiza la clasificación en el espacio de embeddings.

Este enfoque es especialmente apropiado cuando se busca usar KNN de forma efectiva en imágenes, mitigando la alta dimensionalidad de los pixeles crudos y mejorando la separabilidad entre clases.


In [ ]:
# ============================================
# Bloque 7: Extractor de embeddings (Transfer Learning)
# ============================================

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

base = EfficientNetB0(
    weights="imagenet",
    include_top=False,
    pooling="avg",
    input_shape=(224, 224, 3)
)
base.trainable = False

@tf.function
def embed_batch(img_batch):
    x = preprocess_input(img_batch)
    return base(x, training=False)

def extract_embeddings(ds):
    feats, labs = [], []
    for batch_imgs, batch_labels in ds:
        emb = embed_batch(batch_imgs)
        feats.append(emb.numpy())
        labs.append(batch_labels.numpy())
    return np.vstack(feats), np.concatenate(labs)

Xtr_emb, ytr = extract_embeddings(ds_train)
Xva_emb, yva = extract_embeddings(ds_val)
Xte_emb, yte = extract_embeddings(ds_test)

print("✅ Embeddings generados:")
print("Train:", Xtr_emb.shape, "Val:", Xva_emb.shape, "Test:", Xte_emb.shape)


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
✅ Embeddings generados:
Train: (104, 1280) Val: (15, 1280) Test: (30, 1280)


# Bloque 8 — Entrenamiento de KNN con ajuste riguroso de hiperparámetros (*GridSearchCV*)



## Objetivo del bloque
Este bloque implementa el **entrenamiento y ajuste sistemático** de un clasificador **K-Nearest Neighbors (KNN)** utilizando como entrada los **embeddings** previamente extraídos de las imágenes.  
El objetivo principal es encontrar la combinación de hiperparámetros que **maximice el desempeño de generalización** mediante:

- un proceso de búsqueda exhaustiva (*grid search*),
- validación cruzada estratificada (*Stratified K-Fold*),
- y una métrica robusta para multiclase desbalanceado (**F1-macro**).

El resultado final es un modelo KNN optimizado (`best_knn`) listo para evaluación en el conjunto de prueba.

---

## Paso 1: Construcción de un *pipeline* (escalamiento + KNN)
Se define un pipeline que encadena dos etapas:

1. **Estandarización de características**
2. **Clasificación con KNN**

### Justificación del escalamiento (por qué es crítico en KNN)
KNN se basa en la distancia entre puntos en el espacio de características.  
Si las variables tienen distintas escalas, la distancia queda dominada por las dimensiones con mayor magnitud, provocando un sesgo artificial en la noción de “vecindad”.

El escalamiento:
- centra y normaliza cada dimensión,
- garantiza que cada componente del embedding contribuya de forma comparable,
- y mejora la estabilidad del método basado en distancias.

### Beneficio metodológico del pipeline
- Evita *data leakage*: el escalador se ajusta **solo con train** dentro de cada fold.
- Asegura reproducibilidad y consistencia al evaluar múltiples configuraciones.
- Facilita extender el flujo con otros modelos manteniendo el mismo preprocesamiento.

---

## Paso 2: Definición del espacio de búsqueda de hiperparámetros (*param_grid*)
Se define una grilla (conjunto discreto) de combinaciones a evaluar. Incluye tres hiperparámetros fundamentales:

### 2.1 Número de vecinos \(k\)
- Controla la complejidad del clasificador.
- \(k\) pequeño:
  - frontera de decisión más irregular,
  - mayor sensibilidad al ruido (*alta varianza*).
- \(k\) grande:
  - frontera más suave,
  - riesgo de subajuste (*alto sesgo*).

Este parámetro representa el compromiso clásico **sesgo–varianza** en KNN.

### 2.2 Esquema de ponderación de vecinos
- **Uniforme**: todos los vecinos votan con el mismo peso.
- **Distancia**: vecinos más cercanos pesan más.

**Interpretación**
- “distance” puede ser ventajoso cuando:
  - existen clases cercanas en el espacio de embeddings,
  - pero la proximidad fina aporta información relevante,
  - o hay densidades distintas por clase.

### 2.3 Métrica de distancia
Se comparan diferentes nociones de cercanía, tales como:
- Euclidiana (L2),
- Manhattan (L1),
- y una formulación general (Minkowski).

**Relevancia**
El rendimiento de KNN depende fuertemente de la geometría del espacio; en embeddings, cambiar la métrica puede alterar significativamente qué puntos son “vecinos”.

---

## Paso 3: Diseño de validación cruzada estratificada (*StratifiedKFold*)
Se configura una validación cruzada en \(K\) particiones (folds), asegurando que cada fold conserve proporciones similares de clases.

### Justificación
- En multiclase con desbalance, folds no estratificados pueden:
  - excluir clases minoritarias en algún fold,
  - producir métricas inestables,
  - y sesgar la selección de hiperparámetros.
- La estratificación incrementa la **validez** y reduce la varianza de la estimación del rendimiento.

La mezcla aleatoria controlada por semilla mejora la representatividad y reproducibilidad.

---

## Paso 4: Configuración de GridSearchCV (búsqueda exhaustiva + evaluación)
Se crea un objeto de búsqueda que:

1. Entrena el pipeline para cada combinación de hiperparámetros.
2. Evalúa cada combinación mediante validación cruzada.
3. Selecciona la combinación con mejor desempeño promedio según la métrica definida.

### Métrica utilizada: F1-macro
El bloque utiliza **F1-macro** como criterio principal.

**Por qué F1-macro es adecuada aquí**
- Calcula F1 por clase y luego promedia asignando **igual peso a cada clase**.
- Es robusta ante desbalance: una clase minoritaria mal clasificada reduce el puntaje, incluso si la accuracy global parece alta.
- Es apropiada cuando el objetivo es desempeño equilibrado entre enfermedades, no solo “ganar” en las clases frecuentes.

---

## Paso 5: Entrenamiento del proceso de búsqueda sobre embeddings de entrenamiento
Se ajusta la grilla usando los embeddings del conjunto de entrenamiento:

- Cada combinación se evalúa en varios folds.
- Dentro de cada fold:
  - el escalador se ajusta usando solo el subconjunto de entrenamiento del fold,
  - el modelo KNN se entrena,
  - y se evalúa en la partición de validación del fold.

**Resultado conceptual**
Se obtiene una estimación del rendimiento promedio y más estable para cada configuración.

---

## Paso 6: Reporte de la mejor configuración encontrada
Una vez completada la búsqueda, se recuperan:

- los hiperparámetros que maximizan el F1-macro promedio,
- el valor de ese F1-macro como estimación de desempeño en validación cruzada.

Esto permite justificar en el informe:
- qué configuración se eligió,
- por qué se eligió (criterio cuantitativo),
- y qué tan consistente fue su desempeño.

---

## Paso 7: Selección del mejor modelo entrenado (listo para test)
Finalmente, se extrae el estimador óptimo como el modelo final.

**Importancia**
- Este modelo representa la mejor configuración según evidencia empírica controlada (CV).
- Queda preparado para el siguiente paso metodológico correcto: **evaluación final en test**, que debe realizarse una sola vez para reportar generalización.

---

## Resultado del bloque
Al finalizar este bloque se obtiene:

1. Un procedimiento de selección de hiperparámetros riguroso y reproducible.
2. La mejor configuración de KNN según validación cruzada estratificada.
3. Un modelo final (`best_knn`) entrenado bajo buenas prácticas, defendible en un contexto académico y alineado con evaluación robusta para multiclase desbalanceado.


In [ ]:
# ============================================
# Bloque 8: KNN + GridSearchCV
# ============================================

pipe = Pipeline(steps=[
    ("scaler", StandardScaler()),        # crítico para distancias en KNN
    ("knn", KNeighborsClassifier())
])

param_grid = {
    "knn__n_neighbors": [3, 5, 7, 9, 11],
    "knn__weights": ["uniform", "distance"],
    "knn__metric": ["minkowski", "euclidean", "manhattan"]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring="f1_macro",   # multiclase robusta al desbalance
    cv=cv,
    n_jobs=-1,
    verbose=1
)

grid.fit(Xtr_emb, ytr)

print("✅ Mejor KNN encontrado:")
print("Mejores params:", grid.best_params_)
print("Mejor F1_macro (CV):", grid.best_score_)

best_knn = grid.best_estimator_


Fitting 5 folds for each of 30 candidates, totalling 150 fits
✅ Mejor KNN encontrado:
Mejores params: {'knn__metric': 'minkowski', 'knn__n_neighbors': 3, 'knn__weights': 'uniform'}
Mejor F1_macro (CV): 1.0


# Bloque 9 — Evaluación final en el conjunto de prueba (Test)



## Objetivo del bloque
Este bloque realiza la **evaluación final de generalización** del modelo seleccionado (`best_knn`) utilizando el conjunto **Test**, el cual se mantuvo separado durante el entrenamiento y la selección de hiperparámetros.

El propósito es reportar un desempeño **honesto y defendible**, midiendo cómo se comporta el clasificador frente a datos **no vistos** durante el ajuste del modelo.

---

## Paso 1: Generación de predicciones sobre embeddings de Test
Se utiliza el modelo KNN optimizado para predecir la clase de cada instancia del conjunto de prueba a partir de los **embeddings** (`Xte_emb`).

**Interpretación**
- Cada embedding representa una imagen en un espacio de características aprendido por la CNN preentrenada.
- El KNN asigna la clase según los vecinos más cercanos en dicho espacio, aplicando la configuración encontrada en GridSearchCV.

El resultado de este paso es un vector `y_pred` con las clases predichas para cada imagen del test.

---

## Paso 2: Cálculo de métricas globales (Accuracy y F1-macro)

### 2.1 Accuracy
Se calcula la **exactitud** como la proporción total de predicciones correctas:

\[
\text{Accuracy}=\frac{\#\text{predicciones correctas}}{\#\text{muestras totales}}
\]

**Ventaja**
- Es fácil de interpretar como porcentaje de aciertos globales.

**Limitación**
- En problemas desbalanceados puede ser engañosa: un modelo puede lograr alta accuracy si domina clases frecuentes, aunque falle en clases minoritarias.

### 2.2 F1-macro
Se calcula el **F1-macro**, que primero computa el F1-score por clase y luego promedia asignando **el mismo peso a cada clase**:

\[
F1_{\text{macro}}=\frac{1}{K}\sum_{k=1}^{K}F1_k
\]

**Por qué es una métrica prioritaria aquí**
- Penaliza con fuerza el mal desempeño en clases minoritarias.
- Es más representativa cuando el objetivo es un rendimiento equilibrado entre enfermedades.

---

## Paso 3: Reporte de métricas globales formateadas
Se imprimen las métricas con formato de cuatro decimales para facilitar:
- comparación entre modelos,
- inclusión directa en el informe,
- y lectura rápida durante la presentación.

---

## Paso 4: Reconstrucción de nombres de clase para interpretación
Se generan los nombres de clases en el orden correcto a partir del codificador de etiquetas (`LabelEncoder`).

**Importancia metodológica**
- Las métricas por clase deben reportarse usando nombres humanos (por ejemplo, `Tomato___Late_blight`) en lugar de índices numéricos.
- Garantiza coherencia entre:
  - el índice interno de la clase,
  - y el nombre reportado en el informe.

---

## Paso 5: Reporte de clasificación por clase (*classification report*)
Se imprime un informe detallado que incluye, para cada clase:

- **Precisión (precision):** de las predicciones hechas como esa clase, cuántas fueron correctas.
- **Recall (exhaustividad):** de las instancias reales de esa clase, cuántas detectó el modelo.
- **F1-score:** balance entre precisión y recall (media armónica).
- **Support:** número real de ejemplos de esa clase en el test.

**Valor analítico**
Este reporte permite identificar clases donde el modelo:
- es **conservador** (alta precisión, bajo recall),
- o es **agresivo** (alto recall, baja precisión),
y detectar enfermedades específicas con bajo desempeño.

---

## Paso 6: Matriz de confusión (*confusion matrix*)
Se construye e imprime la matriz de confusión, donde:

- La **diagonal principal** representa aciertos por clase.
- Los valores **fuera de la diagonal** representan confusiones sistemáticas entre clases.

**Interpretación práctica**
- Permite detectar patrones típicos de error, por ejemplo:
  - confusión entre enfermedades visualmente similares,
  - sesgos hacia clases con mayor frecuencia,
  - o errores consistentes en ciertas clases minoritarias.

La matriz es especialmente útil para el **análisis crítico** requerido por la rúbrica, ya que aporta evidencia concreta para proponer mejoras (más datos, augmentación, ajuste del extractor, etc.).

---

## Resultado del bloque
Al finalizar, se obtiene una evaluación completa y defendible del modelo en datos no vistos:

1. **Test Accuracy** (visión global de aciertos).
2. **Test F1-macro** (métrica robusta para multiclase desbalanceado).
3. **Reporte por clase** (diagnóstico fino de precisión/recall/F1).
4. **Matriz de confusión** (análisis detallado de patrones de error).

Este conjunto de salidas permite comparar modelos, sustentar conclusiones y construir una discusión crítica alineada con estándares académicos y con la rúbrica del proyecto.


In [ ]:
# ============================================
# Bloque 9: Evaluación final en Test
# ============================================

y_pred = best_knn.predict(Xte_emb)

acc = accuracy_score(yte, y_pred)
f1m = f1_score(yte, y_pred, average="macro")

print(f"✅ Test Accuracy : {acc:.4f}")
print(f"✅ Test F1-macro : {f1m:.4f}\n")

target_names = le.inverse_transform(np.arange(len(le.classes_)))
print(classification_report(yte, y_pred, target_names=target_names))

cm = confusion_matrix(yte, y_pred)
print("Matriz de confusión:\n", cm)


✅ Test Accuracy : 1.0000
✅ Test F1-macro : 1.0000

                      precision    recall  f1-score   support

Tomato___Late_blight       1.00      1.00      1.00        30

            accuracy                           1.00        30
           macro avg       1.00      1.00      1.00        30
        weighted avg       1.00      1.00      1.00        30

Matriz de confusión:
 [[30]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


# Bloque 10 — Inferencia interactiva: subir una imagen y predecir



## Objetivo del bloque
Este bloque implementa un flujo de **inferencia en tiempo real** dentro de Google Colab, permitiendo que el usuario:

1. **Suba una imagen nueva** (no necesariamente perteneciente al dataset).
2. Aplique el mismo preprocesamiento usado en el entrenamiento.
3. Genere su **embedding** con el extractor basado en *Transfer Learning*.
4. Use el clasificador **KNN** ya entrenado para:
   - predecir la clase más probable,
   - y obtener un ranking **Top-5** con probabilidades asociadas.

Este bloque representa una aproximación conceptual a un **despliegue** (*deployment*) en un entorno real: entrada → modelo → salida.

---

## Paso 1: Importación de herramientas para interacción en Colab
Se cargan utilidades propias del entorno Google Colab y de Keras para:

- permitir la **carga manual de archivos** desde el computador del usuario,
- leer y transformar imágenes en arreglos numéricos compatibles con el pipeline.

**Relevancia**
Este paso hace que el notebook no sea solo “entrenamiento”, sino también una herramienta práctica de uso, alineada con la sección de despliegue conceptual del proyecto.

---

## Paso 2: Subida de imagen(s) por parte del usuario
Se habilita un mecanismo interactivo para seleccionar archivos locales.  
El resultado es una colección de archivos cargados, que puede contener una o varias imágenes.

**Interpretación**
- Cada archivo cargado representa una nueva instancia \(x_{\text{nuevo}}\).
- Estas imágenes no requieren estar dentro de la estructura de carpetas por clase.

---

## Paso 3: Definición de una función reutilizable de predicción
Se define una función que encapsula el proceso completo de inferencia para una sola imagen, lo cual es una buena práctica porque:

- hace el flujo modular y reutilizable,
- permite evaluar múltiples imágenes con la misma metodología,
- evita duplicación de código,
- y facilita su futura migración a una API o aplicación.

---

## Paso 4: Carga de la imagen y estandarización del formato de entrada
Dentro de la función:

1. La imagen se carga desde disco.
2. Se redimensiona al tamaño esperado por el pipeline (**mismo tamaño usado en el extractor**).
3. Se convierte a un arreglo numérico (tensor/array).

**Justificación**
Los modelos preentrenados y los extractores de embeddings exigen un tamaño de entrada fijo.  
Además, convertir a arreglo numérico es necesario porque el modelo trabaja con tensores, no con archivos.

---

## Paso 5: Preparación del batch (dimensión adicional)
Se añade una dimensión extra para representar el tamaño del lote (*batch*), aun cuando se prediga una sola imagen.

**Por qué es necesario**
- TensorFlow/Keras espera entradas con forma:
  \[
  (batch\_size, height, width, channels)
  \]
- Esto permite que el mismo pipeline funcione tanto para una imagen como para cientos en lote.

---

## Paso 6: Extracción del embedding con el modelo preentrenado
La imagen procesada se pasa por la función de embeddings.

**Interpretación conceptual**
- La CNN actúa como una transformación:
  \[
  \text{imagen} \rightarrow \text{vector de características}
  \]
- Ese vector representa una descripción compacta (texturas, manchas, estructuras) útil para comparar similitud entre hojas.

Este paso es el puente entre Deep Learning (representación) y KNN (clasificación basada en vecindad).

---

## Paso 7: Predicción de clase con KNN
Con el embedding generado:

1. Se obtiene la **clase predicha** (la más probable según KNN).
2. Se obtienen las **probabilidades por clase**.

**Interpretación**
- En KNN, las probabilidades suelen derivarse de la proporción de vecinos por clase (o ponderaciones), por lo que:
  - reflejan “qué tan apoyada” está la decisión por ejemplos cercanos,
  - y entregan una medida práctica de confianza relativa.

---

## Paso 8: Decodificación de la clase predicha a su nombre original
Como el modelo trabaja con etiquetas numéricas internas, se reconstruye el nombre de clase original (por ejemplo, `Tomato___Leaf_Mold`).

**Importancia**
- Hace la salida interpretable para el usuario final.
- Permite que la herramienta sea utilizable sin conocer la codificación numérica.

---

## Paso 9: Construcción del ranking Top-5
Se ordenan las probabilidades de mayor a menor y se seleccionan las 5 clases más probables.

**Valor práctico**
- El Top-5 es útil cuando:
  - hay clases visualmente similares,
  - el modelo tiene incertidumbre,
  - se busca apoyar a un experto humano con alternativas plausibles.
- Favorece el análisis crítico: si el Top-1 falla, se puede observar si la respuesta correcta estaba en el Top-5.

---

## Paso 10: Inferencia sobre todas las imágenes cargadas
Finalmente, se recorre cada archivo subido y se ejecuta el proceso de predicción, mostrando:

- la clase predicha (Top-1),
- y el ranking Top-5 con probabilidades.

**Resultado operacional**
Este bloque convierte el notebook en una herramienta interactiva de predicción, alineada con un flujo de despliegue conceptual:

**Entrada (imagen) → Embedding → Clasificador KNN → Salida (predicción + ranking)**

---

## Resultado del bloque
Al finalizar este bloque, se obtiene:

1. Un mecanismo funcional para **probar el modelo con imágenes nuevas**.
2. Predicciones interpretables en términos de nombres de enfermedades.
3. Un ranking Top-5 que aporta:
   - interpretabilidad práctica,
   - manejo de incertidumbre,
   - y evidencia para discusión metodológica y de despliegue.


In [ ]:
# ============================================
# Bloque 10: Inferencia (subir imagen y predecir)
# ============================================

from google.colab import files
from tensorflow.keras.preprocessing import image as kimage

uploaded = files.upload()

def predict_image(path):
    img = kimage.load_img(path, target_size=IMG_SIZE)
    arr = kimage.img_to_array(img)
    arr = np.expand_dims(arr, axis=0).astype(np.float32)

    emb = embed_batch(arr).numpy()
    pred = best_knn.predict(emb)[0]
    proba = best_knn.predict_proba(emb)[0]

    pred_label = le.inverse_transform([pred])[0]
    top5_idx = np.argsort(proba)[::-1][:5]
    top5 = [(le.inverse_transform([i])[0], float(proba[i])) for i in top5_idx]

    return pred_label, top5

for fname in uploaded.keys():
    label, top5 = predict_image(fname)
    print("✅ Predicción:", label)
    print("Top-5:")
    for cls, p in top5:
        print(f"  - {cls}: {p:.4f}")


Saving 01425d17-4c97-46e3-b395-c1453b78ab78___GHLB2 Leaf 9100.JPG to 01425d17-4c97-46e3-b395-c1453b78ab78___GHLB2 Leaf 9100.JPG
✅ Predicción: Tomato___Late_blight
Top-5:
  - Tomato___Late_blight: 1.0000
